In [52]:
import sys
import math

In [53]:
#N>=1
N = input('Enter No Of Elements Per Vectors (N): ')

Enter No Of Elements Per Vectors (N): 7


In [54]:
#HRR>=2
HRR = input('Enter HRR (empty for 2): ')

Enter HRR (empty for 2): 3


In [55]:
N = int(N)
if N<1:
    N=1
if not HRR:
    HRR=2
else:
    HRR = int(HRR)
    if HRR<2:
        HRR=2
    if HRR>N:
        HRR=N
NOEPSMin = math.ceil(N/HRR);
NOEPSMax = math.ceil(N/(HRR-1))-1;
NOEPSMM = (NOEPSMin!=NOEPSMax)

In [56]:
#>=NOIPSMin, <=NOIPSMax
if NOEPSMM:
    NOEPSi = input('Enter No Of Elements Per Series (>={}, <={}, empty for {}): '.format(NOEPSMin, NOEPSMax, NOEPSMin))

In [57]:
#RS>=2
RS = input('Enter No Of Vector Repeat Times (empty for 1): ')

Enter No Of Vector Repeat Times (empty for 1): 4


In [58]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH Default Value (empty for 10): ')

Enter IN_WIDTH Default Value (empty for 10): 


In [59]:
Enable_Output_Registers = input('Enable Output Registers? (empty or 0 to disable, 1 to enable): ')

Enable Output Registers? (empty or 0 to disable, 1 to enable): 1


In [60]:
Enable_Input_Latch = input('Enable Input Latch? (empty or 0 to disable, 1 to enable): ')

Enable Input Latch? (empty or 0 to disable, 1 to enable): 1


In [61]:
if HRR>1 and NOEPSMM:
    if not NOEPSi:
        NOEPS=NOEPSMin
    else:
        NOEPSi = int(NOEPSi)
        if NOEPSi<NOEPSMin:
            NOEPS=NOEPSMin
        elif NOEPSi>NOEPSMax:
            NOEPS=NOEPSMax
        else:
            NOEPS=NOEPSi
else:
    NOEPS=NOEPSMin

if not RS:
    RS=2
else:
    RS = int(RS)
    if RS<2:
        RS=2

if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if IN_WIDTH<1:
        IN_WIDTH=1

if not Enable_Output_Registers:
    Enable_Output_Registers=0
else:
    Enable_Output_Registers = int(Enable_Output_Registers)

if not Enable_Input_Latch:
    Enable_Input_Latch=0
else:
    Enable_Input_Latch = int(Enable_Input_Latch)

In [62]:
__Print_To_File = 0

if __Print_To_File<=0:
    of=sys.stdout
else:
    filename="./OTItoSeries_{}_HRx{}_S{}E_RTx{}_".format(N, HRR, NOEPS, RS)
    if Enable_Output_Registers<=0:
        filename+="N"
    filename+="OR_"
    if Enable_Input_Latch<=0:
        filename+="N"
    filename+="IL.v"
    of=open(filename, 'w+')        

In [63]:
print("`timescale 1ns / 1ps\n", file=of)
print("module OTItoSeries_{}_HRx{}_S{}E_RTx{}_".format(N, HRR, NOEPS, RS), end='', file=of)
if Enable_Output_Registers<=0:
    print("N", end='', file=of)
print("OR_", end='', file=of)
if Enable_Input_Latch<=0:
    print("N", end='', file=of)
print("IL", file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {}".format(IN_WIDTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
print("output reg readyForNewDataSeries = 1,", file=of)
lghrr=math.ceil(math.log2(HRR))
if lghrr==1:
    print("output reg inSeries = 0,", file=of)
elif lghrr>1:
    print("output reg [{}:0] inSeries = 0,".format(lghrr-1), file=of)
if lgRS<=1:
    print("output reg repeatTimeIn = 0,", file=of)
else:
    print("output reg [{}:0] repeatTimeIn = 0,".format(lgRS-1), end='', file=of)

print("input inReady,", file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("A{}, ".format(i), end='', file=of)
print(file=of)
NE = N - NOEPS*(HRR-1)
while NE<=0:
    NE += NOEPS
if NE==1:
    print("output reg O0outReady,", file=of)
else:
    print("output reg O0toO{}outReady,".format(NE-1), file=of)
if NE==NOEPS:
    print("output reg ONoutReady=0, //not used", file=of)
elif NE==NOEPS-1:
    print("output reg O{}outReady,".format(NE), file=of)
else:
    print("output reg O{}toO{}outReady,".format(NE, NOEPS-1), file=of)
print("output ", end='', file=of)
if Enable_Output_Registers>0:
    print("reg ", end='', file=of)
if lghrr==1:
    print("outSeries", end='', file=of)
elif lghrr>1:
    print("[{}:0] outSeries".format(lghrr-1)  end='',, file=of)
if Enable_Output_Registers>0:
    print(" = {},".format(HRR-1), file=of)
else:
    print(",", file=of)
print("output ", end='', file=of)
if Enable_Output_Registers>0:
    print("reg ", end='', file=of)
lgRS=math.ceil(math.log2(RS))
if lgRS<=1:
    print("repeatTimeOut", end='', file=of)
else:
    print("[{}:0] repeatTimeOut".format(lgRS-1), end='', file=of)
if Enable_Output_Registers>0:
    print(" = {},".format(RS-1), file=of)
else:
    print(",", file=of)
print("output reg signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(NOEPS-1):
    print("O{}, ".format(i), end='', file=of)
print("O{}".format(NOEPS-1), end='', file=of)
if Enable_Output_Registers>0:
    print(",", file=of)
    if NE==1:
        print("output O0earlyOutReady,", file=of)
    else:
        print("output O0toO{}earlyOutReady,".format(NE-1), file=of)
    if NE==NOEPS:
        print("output reg ONearlyOutReady=0 //not used", file=of)
    elif NE==NOEPS-1:
        print("output O{}earlyOutReady".format(NE), file=of)
    else:
        print("output O{}toO{}earlyOutReady".format(NE, NOEPS-1), file=of)
else:
    print(file=of)
print(");\n", file=of)

`timescale 1ns / 1ps

module OTItoSeries_7_HRx3_S3E_RTx4_OR_IL
#(
parameter IN_WIDTH = 10
)(
input clk, reset, enable,
output reg readyForNewDataSeries = 1,
output reg [1:0] inSeries = 0,
output reg repeatTimeIn = 0,
input inReady,
input signed [IN_WIDTH-1:0] A0, A1, A2, A3, A4, A5, A6, 
output reg O0outReady,
output reg O1toO2outReady,
output reg [{}:0] outSeries = 2,
output reg [1:0] repeatTimeOut = 3,
output reg signed [IN_WIDTH:0] O0, O1, O3,
output O0earlyOutReady,
output O1toO2earlyOutReady
);



In [64]:
print("always @(posedge clk) begin", file=of)
print("\tif(reset) begin", file=of)
print("\t\treadyForNewDataSeries <= 1;", file=of)
print("\t\tinSeries <= 0;", file=of)
print("\tend", file=of)
print("\telse if(enable) begin", file=of)
print("\t\tif(inSeries==0) begin", file=of)
print("\t\t\tif(inReady) begin", file=of)
print("\t\t\t\tif(repeatTimeIn==0) begin", file=of)
print("\t\t\t\t\treadyForNewDataSeries <= 0;", file=of)
print("\t\t\t\tend", file=of)
print("\t\t\t\tinSeries <= 1;", file=of)
print("\t\t\tend", file=of)
print("\t\tend", file=of)
print("\t\telse", end='', file=of)
if HRR>2:
    print(" if(inSeries=={})".format(HRR-1), end='', file=of)
print(" begin", file=of)
print("\t\t\tif(repeatTimeIn=={}) begin".format(RS-1), file=of)
print("\t\t\t\treadyForNewDataSeries <= 1;", file=of)
print("\t\t\t\trepeatTimeIn <= 0;", file=of)
print("\t\t\tend", file=of)
print("\t\t\telse begin", file=of)
if RS==2:
    print("\t\t\t\trepeatTimeIn <= 1;", file=of)
else:
    print("\t\t\t\trepeatTimeIn <= repeatTimeIn+1;", file=of)
print("\t\t\tend", file=of)
print("\t\t\tinSeries <= 0;", file=of)
print("\t\tend", file=of)
if HRR>2:
    print("\t\telse begin", file=of)
    if HRR==3:
        print("\t\t\tinSeries <= 2;", file=of)        
    else:
        print("\t\t\tinSeries <= inSeries+1;", file=of)
    print("\t\tend", file=of)
print("\tend", file=of)
print("end", file=of)
print(file=of)

always @(posedge clk) begin
	if(reset) begin
		readyForNewDataSeries <= 1;
		inSeries <= 0;
	end
	else if(enable) begin
		if(inSeries==0) begin
			if(inReady) begin
				if(repeatTimeIn==0) begin
					readyForNewDataSeries <= 0;
				end
				inSeries <= 1;
			end
		end
		else if(inSeries==2) begin
			if(repeatTimeIn==3) begin
				readyForNewDataSeries <= 1;
				repeatTimeIn <= 0;
			end
			else begin
				repeatTimeIn <= repeatTimeIn+1;
			end
			inSeries <= 0;
		end
		else begin
			inSeries <= 2;
		end
	end
end



In [73]:
if Enable_Input_Latch>0:
    print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(N-1):
        print("A{}l, ".format(i), end='', file=of)
    print("A{}l;".format(N-1), file=of)
    print("always @(posedge clk) begin", file=of)
    print("\tif(enable) begin", file=of)
    print("\t\tif(inReady & (repeatTimeIn==0) & (inSeries==0)) begin", file=of)
    print("\t\t\t", end='', file=of)
    for i in range(N):
        print("A{}l <= A{}; ".format(i, i), end='', file=of)
    print("\n\t\tend", file=of)
    print("\tend", file=of)
    print("end", file=of)
    print(file=of)

reg signed [IN_WIDTH-1:0] A0l, A1l, A2l, A3l, A4l, A5l, A6l;
always @(posedge clk) begin
	if(enable) begin
		if(inReady & (repeatTimeIn==0) & (inSeries==0)) begin
			A0l <= A0; A1l <= A1; A2l <= A2; A3l <= A3; A4l <= A4; A5l <= A5; A6l <= A6; 
		end
	end
end



In [66]:
#(inSeries==0) ? inReady : 1
print("wire eOR = ((inSeries==0) & inReady) | (inSeries!=0);", file=of)
if NE!=NOEPS:
    if HRR==2:
        print("wire eORC = (inSeries==0) & inReady;".format(HRR-1), file=of)
    elif HRR==3:
        print("wire eORC = ((inSeries==0) & inReady) | (inSeries==1);".format(HRR-1), file=of)
    else:
        print("wire eORC = ((inSeries==0) & inReady) | ((inSeries!=0) & (inSeries!={}));".format(HRR-1), file=of)

wire eOR = ((inSeries==0) & inReady) | (inSeries!=0);
wire eORC = ((inSeries==0) & inReady) | (inSeries==1);


In [72]:
if Enable_Output_Registers>0:
    print("always @(posedge clk) if(enable) begin", file=of)
else:
    print("always @(*) begin", file=of)
print("\tcase(outSeries)", file=of)
print("\t\t0: if(inReady & (repeatTimeIn==0)) begin\n\t\t\t", end='', file=of)
for j in range(NOEPS):
    print("O{} ".format(j), end='', file=of)
    if Enable_Output_Registers>0:
        print("<", end='', file=of)
    print("= A{}; ".format(j), end='', file=of)
print(file=of)
for i in range(HRR):
    if i==0:
        print("\t\telse begin", file=of)
    else:
        print("\t\t{}: begin".format(i), file=of)
    print("\t\t\t", end='', file=of)
    for j in range(NOEPS):
        print("O{} ".format(j), end='', file=of)
        if Enable_Output_Registers>0:
            print("<", end='', file=of)
        k=NOEPS*i+j
        if k >= N:
            print("= 0; ", end='', file=of)
        else:
            print("= A{}".format(k), end='', file=of)
            if Enable_Input_Latch>0:
                print("l", end='', file=of)
            print("; ", end='', file=of)
    print(file=of)
    print("\t\tend", file=of)
print("\tendcase", file=of)

if NE==1:
    print("\tO0outReady ", end='', file=of)
else:
    print("\tO0toO{}outReady ".format(NE-1), end='', file=of)
if Enable_Output_Registers>0:
    print("<", end='', file=of)
print("= eOR;", file=of)

if NE!=NOEPS:
    if NE==NOEPS-1:
        print("\tO{}outReady ".format(NE), end='', file=of)
    else:
        print("\tO{}toO{}outReady ".format(NE, NOEPS-1), end='', file=of)
    if Enable_Output_Registers>0:
        print("<", end='', file=of)
    print("= eORC;", file=of)
print("end\n", file=of)

always @(posedge clk) if(enable) begin
	case(outSeries)
		0: if(inReady & (repeatTimeIn==0)) begin
			O0 <= A0; O1 <= A1; O2 <= A2; 
		else begin
			O0 <= A0l; O1 <= A1l; O2 <= A2l; 
		end
		1: begin
			O0 <= A3l; O1 <= A4l; O2 <= A5l; 
		end
		2: begin
			O0 <= A6l; O1 <= 0; O2 <= 0; 
		end
	endcase
	O0outReady <= eOR;
	O1toO2outReady <= eORC;
end



In [68]:
if Enable_Output_Registers>0:
    if NE==1:
        print("assign O0earlyOutReady = eOR;", file=of)
    else:
        print("assign O0toO{}earlyOutReady = eOR;".format(NE-1), file=of)
    if NE==NOEPS-1:
        print("assign O{}earlyOutReady = eORC;".format(NE), file=of)
    elif NE!=NOEPS:
        print("assign O{}toO{}earlyOutReady = eORC;".format(NE, NOEPS-1), file=of)

assign O0earlyOutReady = eOR;
assign O1toO2earlyOutReady = eORC;


In [69]:
if Enable_Output_Registers<=0:
    print("assign outSeries = inSeries;", file=of)
    print("assign repeatTimeOut = repeatTimeIn;", file=of)
else:
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\toutSeries <= {};".format(HRR-1), file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print("\t\tif((outSeries==0) & inReady) begin", file=of)
    print("\t\t\toutSeries <= 1;", file=of)
    print("\t\tend", file=of)
    print("\t\telse", end='', file=of)
    if HRR>2:
        print(" if(outSeries=={})".format(HRR-1), end='', file=of)
    print(" begin", file=of)
    print("\t\t\tif(repeatTimeOut=={}) begin".format(RS-1), file=of)
    print("\t\t\t\trepeatTimeOut <= 0;", file=of)
    print("\t\t\tend", file=of)
    print("\t\t\telse begin", file=of)
    if RS==2:
        print("\t\t\t\trepeatTimeOut <= 1;", file=of)
    else:
        print("\t\t\t\trepeatTimeOut <= repeatTimeOut+1;", file=of)
    print("\t\t\tend", file=of)
    print("\t\t\toutSeries <= 0;", file=of)
    print("\t\tend", file=of)
    if HRR>2:
        print("\t\telse begin", file=of)
        if HRR==3:
            print("\t\t\toutSeries <= 2;", file=of)        
        else:
            print("\t\t\toutSeries <= outSeries+1;", file=of)
        print("\t\tend", file=of)
    print("\tend", file=of)
    print("end", file=of)
    print(file=of)

always @(posedge clk) begin
	if(reset) begin
		outSeries <= 2;
	end
	else if(enable) begin
		if((outSeries==0) & inReady) begin
			outSeries <= 1;
		end
		else if(outSeries==2) begin
			if(repeatTimeOut==3) begin
				repeatTimeOut <= 0;
			end
			else begin
				repeatTimeOut <= repeatTimeOut+1;
			end
			outSeries <= 0;
		end
		else begin
			outSeries <= 2;
		end
	end
end



In [70]:
print("endmodule", file=of)

endmodule


In [71]:
if __Print_To_File>0:
    of.close()